In [1]:
import torch
import numpy as np
import os
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
def load_data(file_name):
    dataMat = []
    labelMat = []
    fr = open(file_name)
    for line in fr.readlines()[1:]:
        line = line.strip().split()
        dataMat.append([line[0], line[1], ' '.join(line[2:-1])])
        labelMat.append(line[-1])
    return dataMat, labelMat

In [3]:
def create_dic(dataMat):
    Dict = []
    for line in dataMat:
        line = line[2].split()
        for element in line:
            if element not in Dict and element not in [',', '.']:
                Dict.append(element)
    return Dict

In [4]:
def create_vector(dict_, dataMat):
    m = len(dataMat)
    n = len(dict_)
    dataMatrix = np.zeros((m, n))
    for i in range(m):
        line = dataMat[i][2].strip().split()
        for word in line:
            if word not in dict_:
                continue
            if word not in [',', '.']:
                dataMatrix[i][dict_.index(word)] = 1
    return dataMatrix

In [5]:
dataMat, labelMat = load_data('train.tsv')
labelMat = [int(x) for x in labelMat]
y = np.zeros((len(dataMat), 5))
for i in range(len(dataMat)):
    y[i][labelMat[i]-1] = 1
testMat, _ = load_data('test.tsv')

In [6]:
if os.path.exists('dict.txt'):
    fr = open('dict.txt', 'r')
    Dict = fr.readline().strip().split()
else:
    Dict = create_dic(dataMat)
    fw = open('dict.txt', 'w')
    fw.write(' '.join(Dict))

dataMatrix = create_vector(Dict, dataMat)
testMatrix = create_vector(Dict, testMat)

In [7]:
print(dataMatrix, testMatrix.shape)

[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]] (66292, 18224)


In [15]:
x = torch.from_numpy(dataMatrix)
y = torch.tensor(labelMat)

In [16]:
print(x.shape, y.shape)

torch.Size([156060, 18224]) torch.Size([156060])


In [17]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.out = torch.nn.Linear(n_hidden, n_output)
        
    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = self.out(x)
        return x

In [18]:
net = Net(n_feature=156060, n_hidden=5000, n_output=5)
print(net)

Net(
  (hidden): Linear(in_features=156060, out_features=5000, bias=True)
  (out): Linear(in_features=5000, out_features=5, bias=True)
)


In [19]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
loss_func = torch.nn.CrossEntropyLoss()

In [14]:
for t in range(100):
    out = net(x)
    loss = loss_func(out, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(t, ': ', loss)

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'